<a href="https://colab.research.google.com/github/ZzangJay/team_project/blob/kimjisan/221112_extra_feature.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
#import tensorflow as tf 
import os
import sys
from datetime import datetime, timedelta
from tqdm.autonotebook import tqdm
import csv
import re
import gzip
import shutil

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  import sys


In [3]:
from google.colab import drive  # for colab
drive.mount('/content/gdrive/')  # for colab
cloud_directory = '/content/gdrive/MyDrive/mimiciv_unzip/'
os.listdir(cloud_directory) # for colab, check cloud directory mount

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


['CHANGELOG.txt',
 'LICENSE.txt',
 'SHA256SUMS.txt',
 'index.html',
 'icu',
 'hosp',
 'drop_2_chartevents.csv']

In [4]:
sepsis = pd.read_csv('/content/gdrive/MyDrive/mimiciv_der/sepsis3.csv', low_memory=False, dtype='str', keep_default_na=False)
urine = pd.read_csv('/content/gdrive/MyDrive/mimiciv_der/urine_output_rate.csv', low_memory=False, dtype='str', keep_default_na=False)
age = pd.read_csv('/content/gdrive/MyDrive/mimiciv_der/age.csv', low_memory=False, dtype='str', keep_default_na=False)
weight = pd.read_csv('/content/gdrive/MyDrive/mimiciv_der/weight_durations.csv', low_memory=False, dtype='str', keep_default_na=False)
vaso=pd.read_csv('/content/gdrive/MyDrive/mimiciv_der/vasoactive_agent.csv', low_memory=False, dtype='str', keep_default_na=False)
admission = pd.read_csv('/content/gdrive/MyDrive/mimiciv_unzip/hosp/admissions.csv', low_memory=False, dtype='str', keep_default_na=False)
patients = pd.read_csv('/content/gdrive/MyDrive/mimiciv_unzip/hosp/patients.csv', low_memory=False, dtype='str', keep_default_na=False)

In [5]:
admission.columns

Index(['subject_id', 'hadm_id', 'admittime', 'dischtime', 'deathtime',
       'admission_type', 'admission_location', 'discharge_location',
       'insurance', 'language', 'marital_status', 'race', 'edregtime',
       'edouttime', 'hospital_expire_flag'],
      dtype='object')

In [8]:
admi_seps = pd.merge(left = admission[['subject_id', 'hadm_id','insurance','race']], right = sepsis[['subject_id','stay_id','sepsis3']], how='inner', left_on='subject_id', right_on='subject_id')
urine = pd.merge(left = urine, right = admi_seps[['subject_id', 'hadm_id','stay_id']], how='inner', left_on='stay_id', right_on='stay_id')
age = pd.merge(left = age, right = admi_seps[['subject_id', 'hadm_id','stay_id']], how='inner', left_on = ['hadm_id','subject_id'], right_on = ['hadm_id', 'subject_id'])


In [10]:
urine.columns

Index(['stay_id', 'charttime', 'weight', 'uo', 'urineoutput_6hr',
       'urineoutput_12hr', 'urineoutput_24hr', 'uo_mlkghr_6hr',
       'uo_mlkghr_12hr', 'uo_mlkghr_24hr', 'uo_tm_6hr', 'uo_tm_12hr',
       'uo_tm_24hr', 'subject_id', 'hadm_id'],
      dtype='object')

In [11]:
weight=urine[['subject_id', 'hadm_id','stay_id', 'weight']]

In [14]:
extra_features = pd.merge(left = weight, right = age[['subject_id', 'hadm_id','age']], how='inner', left_on=['subject_id', 'hadm_id'], right_on=['subject_id', 'hadm_id'])
extra_features.sort_values(by=['hadm_id','stay_id','age'])


,subject_id,hadm_id,stay_id,weight,age
35456812,14990224,20000147,30503572,97.6,71
35456813,14990224,20000147,30503572,97.6,71
35456814,14990224,20000147,30503572,97.6,71
35456815,14990224,20000147,30503572,97.6,71
35456816,14990224,20000147,30503572,94,71
...,...,...,...,...,...
21319129,10355856,29999625,36975675,120.8,69
21319130,10355856,29999625,36975675,125,69
21319131,10355856,29999625,36975675,120.8,69
21319132,10355856,29999625,36975675,120.8,69


In [13]:
extra_features

,subject_id,hadm_id,stay_id,weight,age
0,11861582,27370071,35689140,89,53
1,11861582,27370071,35689140,89,53
2,11861582,27370071,35689140,89,53
3,11861582,27370071,35689140,89,53
4,11861582,27370071,35689140,89,53
...,...,...,...,...,...
37015873,17999101,21874598,37005150,67.8,62
37015874,17999101,24735133,37005150,67.8,61
37015875,11514412,27024433,35475099,63.4,23
37015876,11514412,27555293,35475099,63.4,26


In [15]:
extra_features = extra_features.drop_duplicates(subset=['subject_id','hadm_id'], keep='first')

In [18]:
extra_features['age']=pd.to_numeric(extra_features['age'])
extra_features['weight']=pd.to_numeric(extra_features['weight'])

In [20]:
extra_features['weight']=extra_features['weight'].fillna(extra_features['weight'].mean())

In [21]:
extra_features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 105622 entries, 0 to 37015877
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   subject_id  105622 non-null  object 
 1   hadm_id     105622 non-null  object 
 2   stay_id     105622 non-null  object 
 3   weight      105622 non-null  float64
 4   age         105622 non-null  int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 4.8+ MB


In [22]:
admission.columns

Index(['subject_id', 'hadm_id', 'admittime', 'dischtime', 'deathtime',
       'admission_type', 'admission_location', 'discharge_location',
       'insurance', 'language', 'marital_status', 'race', 'edregtime',
       'edouttime', 'hospital_expire_flag'],
      dtype='object')

In [23]:
 extra_features_2= pd.merge(left = extra_features, right = admission[['hadm_id','insurance','race']], how='inner', left_on='hadm_id', right_on='hadm_id')

In [24]:
extra_features_2

,subject_id,hadm_id,stay_id,weight,age,insurance,race
0,11861582,27370071,35689140,89.000000,53,Other,WHITE
1,11861582,27939019,35689140,89.000000,44,Medicare,WHITE
2,13532491,27658791,38172931,48.700000,91,Medicaid,ASIAN
3,18903459,21077260,36623584,62.100000,43,Other,WHITE
4,18903459,21187549,36623584,62.100000,43,Other,WHITE
...,...,...,...,...,...,...,...
105617,17999101,21874598,37005150,67.800000,62,Other,WHITE
105618,17999101,24735133,37005150,67.800000,61,Other,WHITE
105619,11514412,27024433,35475099,63.400000,23,Other,BLACK/AFRICAN AMERICAN
105620,11514412,27555293,35475099,63.400000,26,Other,BLACK/AFRICAN AMERICAN


In [25]:
patients.columns

Index(['subject_id', 'gender', 'anchor_age', 'anchor_year',
       'anchor_year_group', 'dod'],
      dtype='object')

In [41]:
extra_features_3 = pd.merge(left = extra_features_2, right = patients[['subject_id', 'gender']], how='inner', left_on='subject_id', right_on='subject_id')

In [42]:
extra_features_3

,subject_id,hadm_id,stay_id,weight,age,insurance,race,gender
0,11861582,27370071,35689140,89.000000,53,Other,WHITE,M
1,11861582,27939019,35689140,89.000000,44,Medicare,WHITE,M
2,13532491,27658791,38172931,48.700000,91,Medicaid,ASIAN,F
3,18903459,21077260,36623584,62.100000,43,Other,WHITE,F
4,18903459,21187549,36623584,62.100000,43,Other,WHITE,F
...,...,...,...,...,...,...,...,...
105617,17999101,21874598,37005150,67.800000,62,Other,WHITE,M
105618,17999101,24735133,37005150,67.800000,61,Other,WHITE,M
105619,11514412,27024433,35475099,63.400000,23,Other,BLACK/AFRICAN AMERICAN,F
105620,11514412,27555293,35475099,63.400000,26,Other,BLACK/AFRICAN AMERICAN,F


In [43]:
extra_features_3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 105622 entries, 0 to 105621
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   subject_id  105622 non-null  object 
 1   hadm_id     105622 non-null  object 
 2   stay_id     105622 non-null  object 
 3   weight      105622 non-null  float64
 4   age         105622 non-null  int64  
 5   insurance   105622 non-null  object 
 6   race        105622 non-null  object 
 7   gender      105622 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 7.3+ MB


In [44]:
label=['insurance','gender']

from sklearn.preprocessing import LabelEncoder

extra_features_3.loc[:,label] = extra_features_3.loc[:,label].apply(LabelEncoder().fit_transform)


In [45]:
extra_features_3

,subject_id,hadm_id,stay_id,weight,age,insurance,race,gender
0,11861582,27370071,35689140,89.000000,53,2,WHITE,1
1,11861582,27939019,35689140,89.000000,44,1,WHITE,1
2,13532491,27658791,38172931,48.700000,91,0,ASIAN,0
3,18903459,21077260,36623584,62.100000,43,2,WHITE,0
4,18903459,21187549,36623584,62.100000,43,2,WHITE,0
...,...,...,...,...,...,...,...,...
105617,17999101,21874598,37005150,67.800000,62,2,WHITE,1
105618,17999101,24735133,37005150,67.800000,61,2,WHITE,1
105619,11514412,27024433,35475099,63.400000,23,2,BLACK/AFRICAN AMERICAN,0
105620,11514412,27555293,35475099,63.400000,26,2,BLACK/AFRICAN AMERICAN,0


In [46]:
extra_features_4=pd.get_dummies(extra_features_3, columns=['race'])

In [49]:
extra_features_4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 105622 entries, 0 to 105621
Data columns (total 40 columns):
 #   Column                                          Non-Null Count   Dtype  
---  ------                                          --------------   -----  
 0   subject_id                                      105622 non-null  object 
 1   hadm_id                                         105622 non-null  object 
 2   stay_id                                         105622 non-null  object 
 3   weight                                          105622 non-null  float64
 4   age                                             105622 non-null  int64  
 5   insurance                                       105622 non-null  int64  
 6   gender                                          105622 non-null  int64  
 7   race_AMERICAN INDIAN/ALASKA NATIVE              105622 non-null  uint8  
 8   race_ASIAN                                      105622 non-null  uint8  
 9   race_ASIAN - ASIAN INDIAN 

In [51]:
extra_features_4=extra_features_4.drop(columns=['subject_id','stay_id'])

In [52]:
extra_features_4

,hadm_id,weight,age,insurance,gender,race_AMERICAN INDIAN/ALASKA NATIVE,race_ASIAN,race_ASIAN - ASIAN INDIAN,race_ASIAN - CHINESE,race_ASIAN - KOREAN,...,race_PATIENT DECLINED TO ANSWER,race_PORTUGUESE,race_SOUTH AMERICAN,race_UNABLE TO OBTAIN,race_UNKNOWN,race_WHITE,race_WHITE - BRAZILIAN,race_WHITE - EASTERN EUROPEAN,race_WHITE - OTHER EUROPEAN,race_WHITE - RUSSIAN
0,27370071,89.000000,53,2,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,27939019,89.000000,44,1,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,27658791,48.700000,91,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,21077260,62.100000,43,2,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,21187549,62.100000,43,2,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105617,21874598,67.800000,62,2,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
105618,24735133,67.800000,61,2,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
105619,27024433,63.400000,23,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
105620,27555293,63.400000,26,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [53]:
extra_features_4['weight']=extra_features_4['weight']/150
extra_features_4['age']=extra_features_4['age']/100

In [54]:
extra_features_4.to_csv('/content/gdrive/MyDrive/mimiciv preprocessing_js/221112_extra_feature.ipynb.csv',index=False)